<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---

## Stanford Sentiment Treebank - Movie Review Classification Competition
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data using keras Tokenizer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [1]:
#install aimodelshare library
! pip install aimodelshare==0.0.189

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 967.8/967.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 

In [1]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/sst2_competition_data-repository:latest') 


Data downloaded successfully.


In [ ]:
# Set up X_train, X_test, and y_train_labels objects
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

X_train=pd.read_csv("sst2_competition_data/X_train.csv", squeeze=True)
X_test=pd.read_csv("sst2_competition_data/X_test.csv", squeeze=True)

y_train_labels=pd.read_csv("sst2_competition_data/y_train_labels.csv", squeeze=True)

# ohe encode Y data
y_train = pd.get_dummies(y_train_labels)

X_train.head()

0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
2    Singer/composer Bryan Adams contributes a slew...
3                 Yet the act is still charming here .
4    Whether or not you 're enlightened by any of D...
Name: text, dtype: object

##2.   Preprocess data using keras tokenizer / Write and Save Preprocessor function


In [ ]:
# This preprocessor function makes use of the tf.keras tokenizer

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 40)
(1821, 40)


##3. Fit model on preprocessed data and save preprocessor function and model 


In [ ]:
from tensorflow.keras.layers import SimpleRNN, LSTM
from tensorflow.keras.layers import Dense, Embedding,Flatten
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(10000, 16, input_length=40))
model.add(LSTM(32, return_sequences=True, dropout=0.2))
model.add(LSTM(32, dropout=0.2))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 40, 16)            160000    
                                                                 
 lstm_2 (LSTM)               (None, 40, 32)            6272      
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_18 (Dense)            (None, 2)                 66        
                                                                 
Total params: 174,658
Trainable params: 174,658
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
173/173 [===========================

#### Save preprocessor function to local "preprocessor.zip" file

In [ ]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [ ]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [ ]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://rlxjxnoql9.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this specific Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [ ]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [ ]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 1s 9ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 382

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

## 5. Repeat submission process to improve place on leaderboard


In [ ]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

model2 = Sequential()
model2.add(layers.Embedding(10000, 16, input_length=40))
model2.add(layers.Conv1D(32, 4, activation='relu')) 
model2.add(layers.MaxPooling1D(3)) 
model2.add(layers.Conv1D(32, 4, activation='relu'))
model2.add(layers.GlobalMaxPooling1D())
model2.add(Dense(2, activation='softmax'))

model2.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])
history = model2.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/10
44/44 [==============================] - 2s 20ms/step - loss: 0.6886 - acc: 0.6124 - val_loss: 0.7179 - val_acc: 0.1488
Epoch 2/10
44/44 [==============================] - 1s 14ms/step - loss: 0.6825 - acc: 0.6149 - val_loss: 0.7400 - val_acc: 0.1488
Epoch 3/10
44/44 [==============================] - 1s 13ms/step - loss: 0.6766 - acc: 0.6149 - val_loss: 0.7669 - val_acc: 0.1488
Epoch 4/10
44/44 [==============================] - 1s 15ms/step - loss: 0.6714 - acc: 0.6149 - val_loss: 0.7941 - val_acc: 0.1488
Epoch 5/10
44/44 [==============================] - 1s 15ms/step - loss: 0.6673 - acc: 0.6149 - val_loss: 0.8215 - val_acc: 0.1488
Epoch 6/10
44/44 [==============================] - 1s 11ms/step - loss: 0.6644 - acc: 0.6149 - val_loss: 0.8429 - val_acc: 0.1488
Epoch 7/10
44/44 [==============================] - 0s 11ms/step - loss: 0.6624 - acc: 0.6149 - val_loss: 0.8569 - val_acc: 0.1488
Epoch 8/10
44/44 [==============================] - 1s 11ms/step - loss: 0.6608 - a

In [ ]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:

#Submit Model 2: 

#-- Generate predicted y values (Model 2)
prediction_column_index=model2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 0s 3ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 380

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Compare two or more models 
data=mycompetition.compare_models([1, 2], verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params
0,Embedding,"[None, 40, 16]",160000.000000,Embedding,"[None, 40, 16]",160000
1,Flatten,"[None, 640]",0.000000,LSTM,"[None, 40, 32]",6272
2,Dense,"[None, 2]",1282.000000,LSTM,"[None, 32]",8320
3,None,None,nan,Flatten,"[None, 32]",0
4,None,None,nan,Dense,"[None, 2]",66


In [ ]:
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2023-04-18 04:48:04--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-04-18 04:48:04--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-04-18 04:48:04--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [ ]:
! unzip glove.6B.zip.1

Archive:  glove.6B.zip.1
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [ ]:
# Extract embedding data for 100 feature embedding matrix
import os
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [ ]:
# Build embedding matrix
embedding_dim = 100 # change if you use txt files using larger number of features
max_words = 10000
maxlen = 40
word_index = tokenizer.word_index

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow as tf
model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.Embedding(max_words, embedding_dim, input_length=maxlen))
model3.add(tf.keras.layers.Flatten())
model3.add(tf.keras.layers.Dense(32, activation='relu'))
model3.add(tf.keras.layers.Dense(2, activation='sigmoid'))
model3.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_26 (Embedding)    (None, 40, 100)           1000000   
                                                                 
 flatten_5 (Flatten)         (None, 4000)              0         
                                                                 
 dense_25 (Dense)            (None, 32)                128032    
                                                                 
 dense_26 (Dense)            (None, 2)                 66        
                                                                 
Total params: 1,128,098
Trainable params: 1,128,098
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.layers[0].set_weights([embedding_matrix])
model3.layers[0].trainable = False



model3.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model3.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)
model3.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
173/173 [==============================] - 2s 7ms/step - loss: 0.6260 - acc: 0.6499 - val_loss: 0.8037 - val_acc: 0.5043
Epoch 2/10
173/173 [==============================] - 1s 5ms/step - loss: 0.5072 - acc: 0.7469 - val_loss: 0.6408 - val_acc: 0.6857
Epoch 3/10
173/173 [==============================] - 1s 4ms/step - loss: 0.4302 - acc: 0.8027 - val_loss: 0.7822 - val_acc: 0.6113
Epoch 4/10
173/173 [==============================] - 1s 4ms/step - loss: 0.3659 - acc: 0.8412 - val_loss: 0.9526 - val_acc: 0.5672
Epoch 5/10
173/173 [==============================] - 1s 4ms/step - loss: 0.2971 - acc: 0.8766 - val_loss: 0.6740 - val_acc: 0.7030
Epoch 6/10
173/173 [==============================] - 1s 5ms/step - loss: 0.2384 - acc: 0.9082 - val_loss: 1.0340 - val_acc: 0.5896
Epoch 7/10
173/173 [==============================] - 1s 4ms/step - loss: 0.1810 - acc: 0.9400 - val_loss: 0.7118 - val_acc: 0.7305
Epoch 8/10
173/173 [==============================] - 1s 4ms/step - loss: 0.

In [ ]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model3, framework='keras',
                          transfer_learning=True,
                          deep_learning=True)

with open("model3.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
#Submit Model 3: 

#-- Generate predicted y values (Model 3)
prediction_column_index=model3.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model3.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 0s 1ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 389

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

In [ ]:
# Compare two or more models
data=mycompetition.compare_models([1, 2, 3], verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params,Model_3_Layer,Model_3_Shape,Model_3_Params
0,Embedding,"[None, 40, 16]",160000.000000,Embedding,"[None, 40, 16]",160000,Embedding,"[None, 40, 16]",160000.000000
1,Flatten,"[None, 640]",0.000000,LSTM,"[None, 40, 32]",6272,LSTM,"[None, 40, 256]",279552.000000
2,Dense,"[None, 2]",1282.000000,LSTM,"[None, 32]",8320,Flatten,"[None, 10240]",0.000000
3,None,None,nan,Flatten,"[None, 32]",0,Dense,"[None, 2]",20482.000000
4,None,None,nan,Dense,"[None, 2]",66,None,None,nan


**1. Discuss the dataset in general terms and describe why building a predictive model using this data might be practically useful.  Who could benefit from a model like this? Explain.**

SST data contains fully labeled parse tree having the sentimental score for each phrase. Building a predictive model like this could help companies or individuals to understand others' feelings. Restaurant could adjust their business model base on the reviews customer left. Investors could adjust their investment model based on the comments or reactions the public has toward current economy form etc. 

**2. Discuss which models performed better and point out relevant hyper-parameter values for successful models.**

The best model had flatten layers, lstm layers, dense layers. The activation function is also important, probably playing a key role here. I used softmax and tanh.

